Установим набор данных tensorflow для дальнейшей работы с ними

In [20]:
!pip install tensorflow_datasets

Установим необходимые библеотеки и модули для нашего проекта

In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import matplotlib.image as mpimg
from keras.preprocessing import image

Загрузим датасет для обучения нейронной сети (отличить собаку от животных) с Tensorflow. Мы сразу загружаем датасет с соответствующими ярлыкакми {cats , dogs}.

In [6]:
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.D5MK6L_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


Проверяем загрузились ли ярлыки.

In [22]:
class_names = info.features['label'].names
class_names

['cat', 'dog']

Датасет загрузился в наш проект, но не в нашу локальную репозиторию. Поэтому для удобства в коде ниже мы создаем папку cats_and_dogs, train и папки cats и dogs в которые мы будем загружать наши фотографии для обучения. При переносе будет называть наши фотографии в соответствии с ярлыкамии и давать им номер.

In [8]:
for i, example in enumerate(dataset['train']):
  image, label = example
  save_dir = './cats_vs_dogs/train/{}'.format(class_names[label])
  os.makedirs(save_dir, exist_ok=True)

  filename = save_dir + "/" + "{}_{}.jpg".format(class_names[label], i)
  tf.keras.preprocessing.image.save_img(filename, image.numpy())

Теперь мы импортируем все необходимые пакеты для создание модели нашшей нейронной сети.

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

Далее создадим генератор данных на основе генератора данных изображений, задав необходимые для нашего проекта параметры. С помощью рескейла мы разделим наши фото на 255 частей, и с помощью остальных параметров будем всячески поварачивать, сдвигать и тд наше изображение. Это необходимо для того чтобы наши данные не повторялись и нейросеть постоянно получала новую информацию. Также отдаем 20% на валидацию.

Далее создадим генераторы тренировки и валидации на основе нашего основного генератора выше. Всем нашим генераторым укажем размер 150 на 150 и количество фотографий в одном паркете 128 для более глубокого обучения. Т.к  у нас всего 2 класса, то укажем мод класса binary, а подмножества укажем в соответствии с целью генераторов.

In [10]:
datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2, rotation_range=10,
                              width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.10, horizontal_flip=True)

train_generator = datagen.flow_from_directory('/content/cats_vs_dogs/train',
                                              target_size = (150, 150),
                                              batch_size=128,
                                              class_mode='binary',
                                              subset='training')

validation_generator = datagen.flow_from_directory('/content/cats_vs_dogs/train',
                                              target_size = (150, 150),
                                              batch_size=128,
                                              class_mode='binary',
                                              subset='validation')

Found 18611 images belonging to 2 classes.
Found 4651 images belonging to 2 classes.


Далее создадим саму модель нашей нейронной сети, мы будем создавать нейросеть CNN. CNN - это особый тип нейронных сетей, предназначенный для обработки и анализа многомерных данных, таких как изображения. CNN состоят из нескольких слоев: сверточных, пулинг-слоев и полносвязных слоев.

1. Слои Conv2D:
Первый слой: 32 фильтрf, используется ReLU активация. Этот слой обрабатывает входные данные размером 150x150x3, такого разрешения хватит для детального изучения картинки, а также 3 битовый цвет поможет более точно оценить картинку используя полное RGB.
Остальное слои Conv2D с ReLU активацией, и в каждых слоях мы увеличиваем количество фильтров для более углубленных и крупномасштабных признаков.

2. Слои MaxPooling2D:
Используются после каждого сверточного слоя для уменьшения размерности данных, улучшения инвариантности к масштабу и повороту, а также для борьбы с переобучением. Каждый слой подвыборки имеет размер окна 2.

3. Слои BatchNormalization:
Нормализация входного слоя нейронной сети обычно выполняется путем масштабирования данных, подаваемых в функции активации. Например, когда есть признаки со значениями от 0 до 1 и некоторые признаки со значениями от 1 до 1000, то их необходимо нормализовать, чтобы ускорить обучение. Нормализацию данных можно выполнить и в скрытых слоях нейронных сетей, что и делает метод пакетной нормализации.

4. Слои Dropout:
Слой, применяющий случайное отключение некоторых нейронов для борьбы с переобучением. В нашем случае ставим 20% для более точного результата. И в последнем слое возьмем 50% чтобы провести регуляризацию нашей нейронной сети.

5. Слой Flatten:
Flatten используется для конвертации входящих данных в меньшую размерность.

6. Слой Dense:
Полносвязный слой с заданным количеством нейронов. Последний такой слой возвращает номер соответствующего класса.

Функция ReLU (Rectified Linear Unit): это простая, но эффективная функция активации, используемая в нейронных сетях. Она возвращает 0 для отрицательных значений входного сигнала и само значение для положительных. ReLU обладает преимуществами: простота, нелинейность, быстрое обучение и сходимость.

Функция Sigmoid: нелинейная функция преобразует любое входное значение (x) в число между 0 и 1. Используется в выходном слое нейронной сети для бинарной классификации, ведь её значение (близкое к 0 или 1) интерпретируется как вероятность принадлежности к одному из классов.



In [11]:
from keras.backend import batch_normalization
model = Sequential()

# 1-й слой
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# 2-й слой
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# 3-й слой
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Данная строчка показывает из каких слоев состоит наша нейросеть и их характеристики.

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 74, 74, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0

Компилиуем модель используя binary_crossentropy, чтобы использовать метод потерь. Так же используем оптимизатор адам.

Также используем history чтобы позже вывести график для оценки обучении.

Также задаем 50 эпох для обучения и подключаем нашши генераторы.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

Epoch 1/10
  1/146 [..............................] - ETA: 26:59 - loss: 0.6073 - accuracy: 0.6562

KeyboardInterrupt: 

Сохраняем получившуюся модель

In [ ]:
model.save('cats_and_dogs.h5')

NameError: name 'model' is not defined

Загружаем выбранную модель

In [24]:
model_load = tf.keras.models.load_model('cats_and_dogs.h5')

С помощью matplotlib сделаем графики, чтобы увидеть как обучалась наша нейросеть и как показатели точности и потери менялись от эпохи к эпохе.

In [14]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs, acc, "r", label="Тренировка точности")
plt.plot(epochs, val_acc, "b", label="Точность валидации")
plt.title("Точность тренеровки и валидации")

plt.figure()

plt.plot(epochs, loss, "r", label="Потери при тренеровке")
plt.plot(epochs, val_loss, "b", label="Потери при валидации")
plt.title("Потери тренировки и валидации")

plt.legend()
plt.show()

NameError: name 'history' is not defined

Для простоты использования нейросети я написал следующий код. Он запрашивает изображение и пропускает его через нашу модель, после чего выдает нам результат.

In [36]:
#Используя коллаб загрузим фото
uploaded = files.upload()

for fn in uploaded.keys():
  # Предсказание результата
  path = "/content/" + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0] > 0.5:
    print(fn, "is a cat")
  else:
    print(fn, "is a dog")

Saving dog (16).jpg to dog (16) (2).jpg
1/1 [==============================] - 0s 59ms/step
[0.03293396]
dog (16) (2).jpg is a dog
